In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
from fastai.metrics import accuracy
from fastai.basic_data import *
from skimage.util import montage
import pandas as pd
from torch import optim
import re

from utils import *

I take a curriculum approach to training here. I first expose the model to as many different images of whales as quickly as possible (no oversampling) and train on images resized to 224x224.

I would like the conv layers to start picking up on features useful for identifying whales. For that, I want to show the model as rich of a dataset as possible.

I then train on images resized to 448x448.

Finally, I train on oversampled data. Here, the model will see some images more often than others but I am hoping that this will help alleviate the class imbalance in the training data.

In [2]:
import fastai
from fastprogress import force_console_behavior
import fastprogress
fastprogress.fastprogress.NO_BAR = True
master_bar, progress_bar = force_console_behavior()
fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

In [3]:
df = pd.read_csv('../train.csv')
val_fns = {'69823499d.jpg'}

In [4]:
fn2label = {row[1].Image: row[1].Id for row in df.iterrows()}
path2fn = lambda path: re.search('\w*\.jpg$', path).group(0)

In [5]:
name = f'res50-full-train'

In [6]:
SZ = 224
BS = 64
NUM_WORKERS = 12
SEED=0

In [7]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], '../train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='data')
        .normalize(imagenet_stats)
)

In [8]:
%%time

learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();

learn.fit_one_cycle(14, 1e-2)
learn.save(f'{name}-stage-1')

learn.unfreeze()

max_lr = 1e-3
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(24, lrs)
learn.save(f'{name}-stage-2')

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:10<00:00, 9398437.37it/s] 


epoch     train_loss  valid_loss
1         7.556172    1.095479    
2         6.803409    2.701401    
3         6.085203    3.106928    
4         5.230840    0.661968    
5         4.280574    1.174970    
6         3.516786    0.222890    
7         2.646444    0.007958    
8         1.904013    2.486029    
9         1.184974    0.064373    
10        0.731920    0.014410    
11        0.414916    0.000007    
12        0.205693    0.000074    
13        0.125785    0.000093    
14        0.105435    0.000041    
epoch     train_loss  valid_loss
1         0.105492    0.000017    
2         0.123349    0.000099    
3         0.154840    0.000008    
4         0.208754    0.000013    
5         0.250755    0.000130    
6         0.291193    0.000145    
7         0.320904    0.000001    
8         0.312296    0.000006    
9         0.288110    0.000004    
10        0.261304    0.000013    
11        0.216555    0.000002    
12        0.174737    0.000000    
13        0.172770    0.

In [9]:
SZ = 224 * 2
BS = 64 // 4
NUM_WORKERS = 12
SEED=0

In [11]:
data = (
    ImageItemList
        .from_df(df[df.Id != 'new_whale'], '../train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='data')
        .normalize(imagenet_stats)
)

In [12]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();
learn.load(f'{name}-stage-2')
learn.freeze_to(-1)

learn.fit_one_cycle(12, 1e-2 / 4)
learn.save(f'{name}-stage-3')

learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(22, lrs)
learn.save(f'{name}-stage-4')

epoch     train_loss  valid_loss
1         1.236716    0.000023    
2         0.916667    0.000001    
3         1.244157    0.000018    
4         1.739386    0.000000    
5         1.922417    0.000000    
6         1.645968    0.000015    
7         1.505456    0.000002    
8         1.231914    0.000000    
9         1.049673    0.000000    
10        0.744415    0.000000    
11        0.659144    0.000000    
12        0.533434    0.000000    
epoch     train_loss  valid_loss
1         0.531848    0.000000    
2         0.475978    0.000000    
3         0.524216    0.000001    
4         0.595996    0.000000    
5         0.607106    0.000002    
6         0.656443    0.000003    
7         0.746673    0.000000    
8         0.673066    0.000000    
9         0.617077    0.000000    
10        0.627019    0.000007    
11        0.627812    0.000000    
12        0.613193    0.000000    
13        0.596159    0.000000    
14        0.500447    0.000000    
15        0.526968    0.

In [24]:
# with oversampling
df = pd.read_csv('data/oversampled_train_and_val.csv')

In [25]:
data = (
    ImageItemList
        .from_df(df, '../train', cols=['Image'])
        .split_by_valid_func(lambda path: path2fn(path) in val_fns)
        .label_from_func(lambda path: fn2label[path2fn(path)])
        .add_test(ImageItemList.from_folder('../test'))
        .transform(get_transforms(do_flip=False), size=SZ, resize_method=ResizeMethod.SQUISH)
        .databunch(bs=BS, num_workers=NUM_WORKERS, path='data')
        .normalize(imagenet_stats)
)

In [26]:
%%time
learn = create_cnn(data, models.resnet50, lin_ftrs=[2048])
learn.clip_grad();
learn.load(f'{name}-stage-4')
learn.freeze_to(-1)

learn.fit_one_cycle(2, 1e-2 / 4)
learn.save(f'{name}-stage-5')

learn.unfreeze()

max_lr = 1e-3 / 4
lrs = [max_lr/100, max_lr/10, max_lr]

learn.fit_one_cycle(3, lrs)
learn.save(f'{name}-stage-6')

epoch     train_loss  valid_loss
1         1.701043    0.000080    
2         0.661639    0.000065    
epoch     train_loss  valid_loss
1         0.770123    0.001406    
2         0.779335    0.000061    
3         0.623801    0.000064    
CPU times: user 1h 26min 55s, sys: 34min 36s, total: 2h 1min 31s
Wall time: 2h 1min 39s


## Predict

In [16]:
preds, _ = learn.get_preds(DatasetType.Test)

In [17]:
preds = torch.cat((preds, torch.ones_like(preds[:, :1])), 1)

In [18]:
preds[:, 5004] = 0.06

In [19]:
classes = learn.data.classes + ['new_whale']

In [21]:
create_submission(preds, learn.data, name, classes)

In [22]:
pd.read_csv(f'subs/{name}.csv.gz').head()

,Image,Id
0,a35964195.jpg,new_whale w_ffe8693 w_e0b6a42 w_4d06559 w_fd51859
1,43fe9e728.jpg,new_whale w_ffe8693 w_d573a68 w_b26e855 w_de1dc14
2,0359ba961.jpg,new_whale w_ffe8693 w_6332313 w_0a8a451 w_dfbf100
3,7bb7fce17.jpg,new_whale w_ffe8693 w_8eb9c42 w_44c1eef w_c028816
4,39a6f1f53.jpg,new_whale w_ffe8693 w_ee05147 w_bdb3a05 w_df1cc01


In [23]:
pd.read_csv(f'subs/{name}.csv.gz').Id.str.split().apply(lambda x: x[0] == 'new_whale').mean()

1.0

In [19]:
!kaggle competitions submit -c humpback-whale-identification -f subs/{name}.csv.gz -m "{name}"

100%|████████████████████████████████████████| 183k/183k [00:04<00:00, 37.6kB/s]
Successfully submitted to Humpback Whale Identification